## Data I/O:

### Read/Write from Files:

**General syntactical guideline**
- **Read:** pd.read_format
- **Write:** df.to_format

**Raw Files**
- pd.read_csv()
- pd.read_excel()
- pd.read_json()
- pd.read_html()


**Binary Formats**
- pd.read_feather()
- pd.read_pickle()
- pd.read_parquet()

**SQL**
- pd.read_sql()

### Other Parsable Formats:

- Clipboard (read_clipboard, to_clipboard)
- HDF5 (hierarchical data, read_hdf, to_hdf)
- MessagePack (read_msgpack, to_msgpack)
- stata (read_stata, to_stata)
- SAS (read_sas, write not available)
- Google Big Query (read_gbq, to_gbq)

Detailed Documentation: https://pandas.pydata.org/pandas-docs/stable/io.html

Performance comparison of I/O on various formats: https://pandas.pydata.org/pandas-docs/stable/io.html#io-perf

### Important Arguments for Read/Write functions:

## Type Casting:

## Data Access:

## Data Filtering:

## Data Manipulation:

## Aggregate Functions:

## Concatenation and Joins:

## Indexing: